# War fatalities in Russia in 2022 estimated via excess male mortality
## Different model specifications

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('../deaths-by-age-gender-region-year/deaths-by-age-gender-region-year-1990-2023.csv.gz')

df

,Region,Year,Age,Gender,Deaths
0,Алтайский край,1990,0-4 лет,f,264
1,Алтайский край,1990,10-14 лет,f,28
2,Алтайский край,1990,15-19 лет,f,63
3,Алтайский край,1990,20-24 лет,f,67
4,Алтайский край,1990,25-29 лет,f,94
...,...,...,...,...,...
116691,Ярославская область,2023,75-79 лет,m,681
116692,Ярославская область,2023,80-84 лет,m,727
116693,Ярославская область,2023,85 и более,m,733
116694,Ярославская область,2023,Неизвестно,m,5


In [3]:
# Load HIV deaths

df_hiv = pd.read_excel('../deaths-hiv/deaths-hiv.xlsx', skiprows=1, index_col=0)

hiv_m = df_hiv.values[1:, 1::3].astype(int)
hiv_f = df_hiv.values[1:, 2::3].astype(int)

# Getting values for 0 year old by subtracting the sum over 1+ (and unknown) from the total
hiv_m[0] -= hiv_m[1:].sum(axis=0)
hiv_f[0] -= hiv_f[1:].sum(axis=0)

# Load 2023 data
df_hiv_2023 = pd.read_excel('../deaths-hiv/deaths-hiv-2023.xlsx', skiprows=1)

hiv_m_2023 = df_hiv_2023[98:].values[0, 2:] # includes 0 year old
hiv_m_2023[hiv_m_2023 == '-'] = 0
hiv_m_2023 = hiv_m_2023.astype(int)

hiv_f_2023 = df_hiv_2023[195:].values[0, 2:]
hiv_f_2023[hiv_f_2023 == '-'] = 0
hiv_f_2023 = hiv_f_2023.astype(int)

# Combine
hiv_m = np.concatenate((hiv_m, hiv_m_2023[:, np.newaxis]), axis=1)
hiv_f = np.concatenate((hiv_f, hiv_f_2023[:, np.newaxis]), axis=1)

# Summing over 5-year bands
hiv_m = hiv_m[:-2].reshape(-1, 5, hiv_m.shape[-1]).sum(axis=1)
hiv_f = hiv_f[:-2].reshape(-1, 5, hiv_f.shape[-1]).sum(axis=1)

hiv_years = np.arange(2006, 2024)

In [4]:
region = 'Российская Федерация'

agegroups = ['15-19 лет', '20-24 лет', '25-29 лет',
       '30-34 лет', '35-39 лет', '40-44 лет', '45-49 лет', 
       '50-54 лет', '55-59 лет']

LinReg = LinearRegression()

excess = np.zeros((len(agegroups), 2, 2, 3, 2))
uncert = np.zeros_like(excess)

for h, hiv in enumerate([False, True]):
    for i, age in enumerate(agegroups):
        # Get the data
        male = df[
            (df.Region == region) & 
            (df.Age == age) & 
            (df.Gender == 'm') & 
            (df.Year >= 2006)
        ][['Year', 'Deaths']].values

        female = df[
            (df.Region == region) & 
            (df.Age == age) & 
            (df.Gender == 'f') & 
            (df.Year >= 2006)
        ][['Year', 'Deaths']].values

        if hiv:
            male[:,1] -= hiv_m[3 + i, hiv_years >= 2006]
            female[:,1] -= hiv_f[3 + i, hiv_years >= 2006]

        ratio = male[:,1] / female[:,1]
        x = np.arange(2006, 2024)
        
        for j, fit_length in enumerate([1, 5, 10]):
            beg = fit_length + 4

            # Perform linear fit and extrapolate        
            LinReg.fit(x[-beg:-4].reshape(-1,1), ratio[-beg:-4].reshape(-1,1))
            yhat_sklearn = LinReg.predict(x[-beg:].reshape(-1,1))
            excess[i, h, 0, j, :] = male[-2:,1] - yhat_sklearn[-2:, 0] * female[-2:,1]
                        
            # Compute the uncertainty
            if fit_length > 1:
                X = np.concatenate((x[-beg:].reshape(-1,1), np.ones((beg,1))), axis=1)
                y = ratio[-beg:].reshape(-1,1)
                beta = np.linalg.pinv(X[-beg:-4].T @ X[-beg:-4]) @ X[-beg:-4].T @ y[-beg:-4]
                yhat = X[-beg:] @ beta
                assert(np.allclose(yhat, yhat_sklearn))

                sigma2 = np.sum((y[-beg:-4] - yhat[-beg:-4])**2) / (y[-beg:-4].size - 2) # sigma^2 = MSE/(n-p)
                S = np.linalg.pinv(X[-beg:-4].T @ X[-beg:-4])
                predictive_var = sigma2 * X[-2:] @ S @ X[-2:].T + sigma2
                predictive_std = np.sqrt(np.diag(predictive_var)) 
                uncert[i, h, 0, j, :] = predictive_std * female[-2:,1]

            # Perform exponential fit and extrapolate
            LinReg.fit(x[-beg:-4].reshape(-1,1), np.log(ratio[-beg:-4] - 1).reshape(-1,1))
            yhat_sklearn = LinReg.predict(x[-beg:].reshape(-1,1))
            yhat_sklearn = np.exp(yhat_sklearn) + 1
            excess[i, h, 1, j, :] = male[-2:,1] - yhat_sklearn[-2:, 0] * female[-2:,1]
            
            # Compute the uncertainty
            if fit_length > 1:
                X = np.concatenate((x[-beg:].reshape(-1,1), np.ones((beg,1))), axis=1)
                y = np.log(ratio[-beg:] - 1).reshape(-1,1)
                beta = np.linalg.pinv(X[-beg:-4].T @ X[-beg:-4]) @ X[-beg:-4].T @ y[-beg:-4]
                yhat = X[-beg:] @ beta
                assert(np.allclose(yhat, np.log(yhat_sklearn - 1)))

                sigma2 = np.sum((y[-beg:-4] - yhat[-beg:-4])**2) / (y[-beg:-4].size - 2) # sigma^2 = MSE/(n-p)
                S = np.linalg.pinv(X[-beg:-4].T @ X[-beg:-4])
                predictive_var = sigma2 * X[-2:] @ S @ X[-2:].T + sigma2
                predictive_std = np.sqrt(np.diag(predictive_var)) 
                predictive_std = np.max((
                    np.exp(yhat + predictive_std) - np.exp(yhat), 
                    np.exp(yhat) - np.exp(yhat - predictive_std)
                ))
                uncert[i, h, 1, j, :] = predictive_std * female[-2:,1]

print('                                  RAW                 |              HIV subtracted') 
print('                    1lin  5lin 10lin  1exp  5exp 10exp  1lin  5lin 10lin  1exp  5exp 10exp') 
print('------------------------------------------------------------------------------------------')

for i in range(len(excess)):
    print(f'{agegroups[i]}:   ', end='')
    for y in range(2):
        if y == 1:
            print('             ', end='')
        print(f'{2022 + y}: ', end='')
        for h in range(2):
            for e in range(2):
                for j in range(3):
                    print(f'{excess[i,h,e,j,y]:5.0f} ', end='')
        print(f'\n                 +-', end='')
        for h in range(2):
            for e in range(2):
                for j in range(3):
                    print(f' {uncert[i,h,e,j,y]:4.0f} ', end='')
        if y == 0:
            print('')
    print('\n')
        
print('-----------------------------------------------------------------------------------------')
print('15--49:      ', end='')
for y in range(2):
    if y == 1:
        print('             ', end='')
    print(f'{2022 + y}: ', end='')
    for h in range(2):
        for e in range(2):
            for j in range(3):
                print(f'{np.sum(excess[:-2,h,e,j,y]):5.0f} ', end='')
    print(f'\n                 +-', end='')
    for h in range(2):
        for e in range(2):
            for j in range(3):
                print(f' {np.sqrt(np.sum(uncert[:-2,h,e,j,y]**2)):4.0f} ', end='')
    print('')

                                  RAW                 |              HIV subtracted
                    1lin  5lin 10lin  1exp  5exp 10exp  1lin  5lin 10lin  1exp  5exp 10exp
------------------------------------------------------------------------------------------
15-19 лет:   2022:   329   353   292   329   352   293   331   337   273   331   337   275 
                 +-    0   165   151     0   202   165     0   173   155     0   213   168 
             2023:   -98   -52  -124   -98   -55  -123  -104   -78  -154  -104   -79  -153 
                 +-    0   194   164     0   209   170     0   205   169     0   220   174 

20-24 лет:   2022:  2610  3069  2681  2610  3022  2681  2438  2868  2406  2438  2828  2416 
                 +-    0   376   416     0   477   491     0   465   429     0   585   494 
             2023:  2694  3281  2829  2694  3211  2822  2502  3057  2511  2502  2995  2519 
                 +-    0   430   439     0   478   492     0   533   455     0   588   49

In [5]:
for i in range(len(excess)):
    print(f'{agegroups[i]}:   ', end='')
    for y in range(2):
        if y == 1:
            print('             ', end='')
        print(f'{2022 + y}: ', end='')
        print(f'{excess[i,0,0,1,y]:5.0f} +-', end='')
        print(f' {uncert[i,0,0,1,y]:4.0f} ', end='')
        if y == 0:
            print('')
    print('\n')
        
print('--------------------------------')
print('15--49:      ', end='')
for y in range(2):
    if y == 1:
        print('             ', end='')
    print(f'{2022 + y}: ', end='')
    print(f'{np.sum(excess[:-2,0,0,1,y]):5.0f} +-', end='')
    print(f' {np.sqrt(np.sum(uncert[:-2,0,0,1,y]**2)):4.0f} ', end='')
    print('')

15-19 лет:   2022:   353 +-  165 
             2023:   -52 +-  194 

20-24 лет:   2022:  3069 +-  376 
             2023:  3281 +-  430 

25-29 лет:   2022:  3961 +-  317 
             2023:  5840 +-  326 

30-34 лет:   2022:  4264 +-  512 
             2023:  6650 +-  531 

35-39 лет:   2022:  6047 +-  796 
             2023: 10691 +-  861 

40-44 лет:   2022:  2692 +-  789 
             2023:  6212 +-  903 

45-49 лет:   2022:  3209 +-  818 
             2023:  7920 +-  924 

50-54 лет:   2022:   990 +- 1190 
             2023:  3042 +- 1345 

55-59 лет:   2022:    83 +-  532 
             2023:  2007 +-  546 

--------------------------------
15--49:      2022: 23594 +- 1568 
             2023: 40542 +- 1738 


In [6]:
for i in range(len(excess)):
    print(f'{agegroups[i]}:   ', end='')
    for y in range(2):
        if y == 1:
            print('             ', end='')
        print(f'{2022 + y}: ', end='')
        print(f'{excess[i,1,0,1,y]:5.0f} +-', end='')
        print(f' {uncert[i,1,0,1,y]:4.0f} ', end='')
        if y == 0:
            print('')
    print('\n')
        
print('--------------------------------')
print('15--49:      ', end='')
for y in range(2):
    if y == 1:
        print('             ', end='')
    print(f'{2022 + y}: ', end='')
    print(f'{np.sum(excess[:-2,1,0,1,y]):5.0f} +-', end='')
    print(f' {np.sqrt(np.sum(uncert[:-2,1,0,1,y]**2)):4.0f} ', end='')
    print('')

15-19 лет:   2022:   337 +-  173 
             2023:   -78 +-  205 

20-24 лет:   2022:  2868 +-  465 
             2023:  3057 +-  533 

25-29 лет:   2022:  3438 +-  193 
             2023:  5385 +-  199 

30-34 лет:   2022:  3395 +-  246 
             2023:  5750 +-  255 

35-39 лет:   2022:  4466 +-  566 
             2023:  9068 +-  606 

40-44 лет:   2022:  3064 +-  722 
             2023:  6995 +-  817 

45-49 лет:   2022:  3049 +-  838 
             2023:  7687 +-  941 

50-54 лет:   2022:  1038 +- 1104 
             2023:  2928 +- 1247 

55-59 лет:   2022:    61 +-  537 
             2023:  1944 +-  549 

--------------------------------
15--49:      2022: 20618 +- 1374 
             2023: 37864 +- 1533 
